# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [1]:
import json
import pandas as pd
import random
from pathlib import Path
from datetime import datetime
from pydantic import BaseModel, Field
from typing import Any
from io import StringIO
from language_models.agents.chain import AgentChain
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.tools.tool import Tool
from language_models.proxy_client import BTPProxyClient
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool
from language_models.settings import settings
from pprint import pprint

In [2]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

Understanding LLMs requires balancing algorithmic reasoning and human thought. Algorithmic reasoning is deterministic, producing consistent outcomes from identical inputs, unlike human thought, which is creative and subjective. LLMs lie between these extremes: they are fluent in natural language but do not truly understand semantics. They can execute algorithms but are limited in proficiency. Practically, executing algorithms involves using tools for desired outcomes. LLMs thus represent a hybrid, processing and generating text while relying on external software for algorithm execution.

LLMs are like new employees who need guidance and tools to perform well. They have potential and language skills but depend on external resources to execute tasks effectively. Providing necessary resources ensures their optimal performance, just as with new hires. Neglecting essential guidance or tools may require adjustments, akin to adapting to the needs of a new employee to ensure success.

To get the best results from LLMs, it's important to craft clear and effective prompts. Prompt engineering is an iterative process. Start with something simple and add more details later. Things to consider:

1. **Be specific:** Provide detailed information to help the LLM understand your query and give tailored responses.
2. **Ask clear questions:** Pose one question at a time to minimize confusion.
3. **Ask follow-up questions:** Clarify incomplete or unclear initial responses with rephrased queries or additional context.
4. **Use full sentences:** Provide comprehensive context with clear and concise sentences.
5. **Provide examples:** Use examples to help the LLM understand your requirements and respond appropriately.

Additionally, consider both system prompts and task prompts:

- **System prompts:** Set general behavior guidelines, such as the persona the LLM should adopt or how it should handle specific tasks and edge cases. For instance, instruct the LLM to check the date before other tasks or respond in a specific format.

- **Task prompts:** For simple queries, a direct question might suffice. For complex tasks, use structured prompts. For example, when classifying an IT ticket, provide clear details about the ticket, user, and date to ensure accurate handling.

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

The following cell defines a movie review prompt, calls GPT-3.5 to analyze the sentiment of the review.

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


Below, we've adjusted the prompt to explicitly request a response indicating whether the sentiment of the movie review is positive or negative. The goal is to receive a concise sentiment analysis without additional text, catering to scenarios where we only need a binary classification of sentiment.

In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


We continue to prompt the model to provide a binary sentiment label for the given movie review. The review is positive, so the expected response should be 1 for positive sentiment. This setup is useful when we want to streamline the process of labeling data for sentiment analysis tasks.

In [6]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative)."""

prompt = """Wow! This movie was incredible. The acting was superb,
and the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


Now, we present an unrelated question to the language model, asking about the weather in Seattle. However, we still provide the system prompt requesting a binary sentiment label for a movie review. This edge case might result in unexpected or irrelevant responses from the language model, demonstrating the importance of providing relevant prompts for accurate analysis.

In [7]:
system_prompt = """Take the following movie review determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative)."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
pprint(response)

("I'm sorry, I am an AI language model and I do not have access to real-time "
 'weather information. I recommend checking a reliable weather website or '
 'using a weather app to get the most accurate and up-to-date forecast for '
 'Seattle.')


To cover edge cases, we provide clear instructions to the language model on how to handle scenarios where the input does not match the expected format. In this case we ask the LLM to respond with -1.

In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

LLMs use various tools to achieve specific goals, streamline operations, and automate tasks. These tools include:

1. **Data retrieval tools:** Extract information from systems or databases using APIs, SDKs, and real-time metrics.
2. **Communication tools:** Facilitate data exchange with external stakeholders via emails, notifications, or alerts.
3. **Data manipulation tools:** Update or modify data within systems, often requiring approval to manage operational impacts.

Specialized tools also exist to handle tasks LLMs struggle with, like performing calculations or accessing current date and time.

In [9]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response) # correct answer: $1,289.98

Total Raw Cost = $1,290.98


In the following code cell, we define a simple calculator function and a Pydantic - you can do it in a different way but for simplicity we will use Pydantic - model Calculator to represent its input arguments. We then create a tool instance calculator_tool using the Tool class from our repository, specifying the calculator function, its name, description, and the Pydantic model for argument validation.

By using Pydantic for JSON model schemas, we ensure that the LLM understands how to use the tools effectively. Additionally, Pydantic allows us to validate the LLM responses easily.

In [10]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)

print(calculator_tool)

tool name: Calculator, tool description: Use this tool when you want to do calculations., tool input: {{'expression': {{'description': 'A math expression.', 'title': 'Expression', 'type': 'string'}}}}


In the system prompt below, we provide clear instructions to the language model on how to utilize the available tools, particularly the calculator tool, to calculate the result accurately based on the given prompt. This is essential for guiding the language model in effectively accessing and leveraging external tools to perform specific tasks, ensuring accurate and helpful responses to user queries.

In this case, we request JSON output from the language model, specifying that it should include a thought, tool, and tool input in its response. This structured format ensures clarity and consistency in the model's outputs, facilitating easy parsing and interpretation. While JSON output is suitable for this demonstration, in practical applications, text responses parsed via regex are often preferred for their simplicity and cost-effectiveness, as they eliminate the need to output JSON formatting elements such as brackets.

In [11]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "You should always think about what to do consider previous and subsequent steps",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


By using the input arguments the LLM provided in its response, we can successfully run software code and show the return value or observation to the LLM.

In [12]:
print(calculator(**response["tool_input"]))

1289.98


Additionally, we introduce an observation to capture the intermediate steps or observations made during the calculation process. Given the model's inability to recall previous conversations, we introduce a mechanism to track and provide previous steps to the model. This ensures that the model can make informed decisions on when to provide the user with the final answer.

It's important to recognize that LLMs can sometimes produce errors in their output, such as generating incorrect JSON or invalid tool input arguments. To address this, it is crucial to highlight these errors to the LLM in hopes that it can correct them. However, the LLM may not always be able to fix its own mistakes. In such instances, after several iterations of feedback and corrections, we may need to provide a fallback response, such as "None" or empty strings.

This iterative process, combined with structured output formats and tool usage, forms the basis of an LLM-powered AI agent. While it may seem like magic in demonstrations, the underlying mechanism is actually quite straightforward.

![ReAct prompting](../img/react.png)

In [13]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}

Observation: tool result
... (this Thought/Tool/Tool Input/Observation can repeat N times)

When you know the answer, you MUST use the following JSON format:
{{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = """Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12

This was your previous work:
Thought: The user wants me to calculate the total raw cost. I will use the Calculator tool.
Tool: Calculator
Tool Input: {"expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"}
Observation: Tool Response: 1289.98"""

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I now know what to respond",
    "tool": "Final Answer",
    "tool_input": "The total raw cost is $1289.98"
}


## 3. Chat-based AI Agents

AI agents are mainly used in two ways, with one key application being chat-based interactions. Users converse with LLMs for tasks in customer support, recruitment, production adjustments, sales forecasting, etc. This mirrors the intuitive way humans think about language.

### Earthquake

The following code snippet configures an LLM to specialize in answering earthquake-related questions by simulating the expertise of a United States Geological Survey (USGS) expert. It integrates various tools, including those for earthquake information and current date retrieval, to enhance its responses. Additionally, the LLM's output is formatted according to a specified schema, ensuring clarity and consistency in its responses. The structured output format of the final answer will become more useful later on.

In [14]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=10,
)

The following question demonstrates how the LLM processes a user's query about the number of earthquakes that occurred on the current day. Using the provided tools, the LLM first retrieves the current date and time. This initial step is crucial for accurately identifying and counting the seismic activities for the day. This process shows the power of the [Chain-of-Thought](https://arxiv.org/abs/2201.11903), particularly the [ReAct](https://arxiv.org/abs/2210.03629) prompting method, which enables the AI to tackle multi-step problems. By breaking down the task into smaller, manageable steps — first obtaining the current date, then querying the relevant seismic data — the LLM can deliver precise and contextually appropriate answers.

In [15]:
response = earthquake_agent.invoke({"question": "How many earthquakes occurred today?"})

30/05/24 13:56:29 INFO Prompt:
How many earthquakes occurred today?
30/05/24 13:56:32 INFO Raw response:
{
  "thought": "I need to get the current date first to determine the start and end times for the earthquake query.",
  "tool": "Current Date",
  "tool_input": {}
}
30/05/24 13:56:32 INFO Thought:
I need to get the current date first to determine the start and end times for the earthquake query.
30/05/24 13:56:32 INFO Tool:
Current Date
30/05/24 13:56:32 INFO Tool input:
{}
30/05/24 13:56:32 INFO Tool response:
2024-05-30 13:56:32.845275
30/05/24 13:56:41 INFO Raw response:
{
  "thought": "Now that I have the current date, I can use it to query the number of earthquakes that occurred today.",
  "tool": "Count Earthquakes",
  "tool_input": {
    "start_time": "2024-05-30T00:00:00",
    "end_time": "2024-05-30T23:59:59"
  }
}
30/05/24 13:56:41 INFO Thought:
Now that I have the current date, I can use it to query the number of earthquakes that occurred today.
30/05/24 13:56:41 INFO Too

In [16]:
print(response.final_answer["content"])

There were 87 earthquakes today.


Now we give the LLM a follow-up question where "Show me 3" refers to the earthquakes that occurred today. By keeping track of the chat history, the LLM can understand the context and continuity of the conversation. This enables the LLM to recognize that "today" is the time frame in question and to provide details about three specific earthquakes that occurred on the current day. This capability to maintain and utilize chat history is essential for coherent and contextually accurate multi-turn interactions.

In [17]:
response = earthquake_agent.invoke({"question": "Show me 3."})

30/05/24 13:56:46 INFO Prompt:
Show me 3.
30/05/24 13:56:51 INFO Raw response:
{
  "thought": "I need to query the earthquakes that occurred today and limit the results to 3.",
  "tool": "Query Earthquakes",
  "tool_input": {"start_time": "2024-05-30T00:00:00", "end_time": "2024-05-30T23:59:59", "limit": 3}
}
30/05/24 13:56:51 INFO Thought:
I need to query the earthquakes that occurred today and limit the results to 3.
30/05/24 13:56:51 INFO Tool:
Query Earthquakes
30/05/24 13:56:51 INFO Tool input:
{'start_time': '2024-05-30T00:00:00', 'end_time': '2024-05-30T23:59:59', 'limit': 3}
30/05/24 13:56:51 INFO Tool response:
{'type': 'FeatureCollection', 'metadata': {'generated': 1717070211000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-05-30T00%3A00%3A00&endtime=2024-05-30T23%3A59%3A59&limit=3&mindepth=-100&maxdepth=1000', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.14.1', 'limit': 3, 'offset': 1, 'count': 3}, 'features': [{'type': 'Fea

In [18]:
print(response.final_answer["content"])

Here are 3 earthquakes that occurred today:

1. A magnitude 2.42 earthquake occurred 2 km SW of Pāhala, Hawaii. [Details](https://earthquake.usgs.gov/earthquakes/eventpage/hv74253361)

2. A magnitude 0.66 earthquake occurred 3 km WSW of The Geysers, CA. [Details](https://earthquake.usgs.gov/earthquakes/eventpage/nc75013317)

3. A magnitude 2.39 earthquake occurred 91 km SSW of Alberto Oviedo Mota, B.C., MX. [Details](https://earthquake.usgs.gov/earthquakes/eventpage/ci40606759)


The LLM can also handle straightforward informational questions that don't require the use of additional tools. In this case, the question about the possibility of MegaQuakes (magnitude 10 or larger) can be answered directly by the language model without invoking any external tools. This shows the LLM's ability to recognize when it can rely on its internal knowledge base to provide an accurate and immediate response, avoiding unnecessary tool usage.

In [19]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

30/05/24 13:57:04 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
30/05/24 13:57:12 INFO Raw response:
{'content': 'Theoretically, yes, an earthquake of magnitude 10 or larger could occur. However, it's highly unlikely. The magnitude of an earthquake is related to the length of the fault on which it occurs - the larger the fault, the larger the earthquake. The simple version is that the biggest fault on Earth, the Pacific Ring of Fire, could potentially create an earthquake of up to magnitude 9.5, but anything larger would likely require a fault that goes around the entire planet, which doesn't exist. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960.'}
30/05/24 13:57:19 INFO Raw response:
{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": {"content": "Theoretically, yes, a magnitude 10 or larger earthquake could occur. However, it's highly unlikely. The largest earthquake ever recorded was a magnitud

In [20]:
pprint(response.final_answer["content"])

('Theoretically, yes, a magnitude 10 or larger earthquake could occur. '
 "However, it's highly unlikely. The largest earthquake ever recorded was a "
 'magnitude 9.5 in Chile in 1960. An earthquake of magnitude 10 would release '
 '31.6 times more energy than the magnitude 9.5 earthquake. Such an event '
 'would require a fault line capable of accommodating such an event, which '
 "does not exist on Earth. Therefore, while it's theoretically possible, it's "
 "practically impossible with what we know about the Earth's crust.")


## 4. Service-based AI Agents

Alternatively, LLMs can be integrated into existing applications as services. For instance, a dashboard button could trigger an LLM to generate sales reports. Unlike traditional APIs, LLMs offer data interpretation and content generation, providing insights and actionable recommendations. This involves encapsulating the AI agent within a function that processes user requests and linking it to an API endpoint. Additionally this allows us to automate workflows based on events.

Almost everything in web or mobile applications involves text in some form. This text can be sent to a language model, which can then use the information to perform tasks such as recommending or comparing selected cars.

### Contract Drafting

When it comes to using LLMs to draft contracts, it's important to consider that contracts are often lengthy documents, sometimes spanning over 100 pages. This renders conversation-based applications impractical, as LLMs cannot generate such extensive documents, and users prefer not to interact directly with the LLM and manage its outputs themselves.

Here's a potential approach for an application to draft contracts: Rather than expecting the LLM to generate the entire document at once, we can break it down into manageable sections. Users could provide bullet points for each section, allowing the LLM to formulate the corresponding paragraphs and suggest a title for each section. The application would then concatenate these outputs to create a comprehensive document. Additionally, in practical scenarios, it's likely necessary to grant the LLM access to specific laws or legal references in some manner.

In [21]:
system_prompt = """You are a corporate lawyer. Take the follow bullet points and generate a draft of a section for a contract. Make it lengthy."""

task_prompt = """Bullet points:
{section}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class ContractSection(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")

contract_drafting_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["section", "section_name"],
    tools=None,
    output_format=ContractSection,
    iterations=3,
)

With the structured output format outlined earlier, the language model will provide us with both a title and the content for each section. Using the capabilities of structured output formats, we aren't restricted to requesting solely string outputs as the final answer; instead, we can ask for more complex data structures.

In [22]:
def generate_contract(contract_sections: list[str]) -> str:
    sections = []
    for contract_section in contract_sections:
        response = contract_drafting_agent.invoke({"section": contract_section})
        section = str(response.final_answer["title"]) + "\n\n" + str(response.final_answer["content"])
        sections.append(section)
        contract_drafting_agent.reset()
    return "\n\n".join(sections)

In [23]:
definitions = "Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA."

amendment = """INVOICING AND PAYMENT TERMS
Clause 12.1(ii) of the GMA shall be cancelled and substituted as follow:
[*****]
[*****]
[*****]

Any other provision of Clause 12 shall remain in full force and effect.

PRICE CONDITIONS
(i) Clause 3.2 of the Exhibit 14 of the GMA shall be cancelled and substituted as follow:
“3.2 Technical conditions for prices adjustment
The prices set out in this Exhibit 14 shall be modified every [*****] at the occasion of the invoicing reconciliation pursuant to Clause 11
(“Reconciliation”) if the Standard Operations of the Aircraft, analyzed at the time of the adjustment (all calculations are made with figures corresponding to [*****], change by more or less
[*****] with respect to the estimated values of the same parameters, considered at the time of commencement of the Term.
As from the date this Agreement enters into force, the Parties agree to take into account the following basic operating parameters (the
“Standard Operations”) as a reference for the above calculation:
[*****]
[*****]
[*****]"""

effective_date_and_duration = "Amendment is effective starting on the date of its signature by both Parties."

confidentiality = """Confidential Information released by either of the Parties (the “Disclosing Party”) to the
other Party (the “Receiving Party”) shall not be released in whole or in part to any third party:
- Not to deliver, disclose or publish it to any third party including subsidiary companies and companies having an interest in its capital
- Use Confidential Information solely for the purpose of this Amendment
- Disclose the Confidential Information only to those of its direct employees
- Not to duplicate the Confidential Information nor to copy

Any Confidential Information shall remain the property of the Disclosing Party.

The Receiving Party hereby acknowledges and recognises that Confidential Information is protected by copyright Laws and related
international treaty provisions, as the case may be.

This shall survive termination or expiry of this Amendment for a period of five (5) years following such End Date."""

governing_law = """Pursuant to and in accordance with Section 5-1401 of the New York General Obligations Law.

Arbitration: in the event of a dispute arising out of or relating to this Amendment, including without limitation disputes regarding the
existence, validity or termination of this Amendment (a “Dispute”), either Party may notify such Dispute to the other through service of a
written notice (the “Notice of Dispute”).

Arbitration, and any proceedings, and meetings incidental to or related to the arbitration process, shall take place in New York.

Arbitration shall be kept confidential and the existence of the proceeding and any element.

During any period of negotiation or arbitration, the Parties shall continue to meet their respective obligations.

Notwithstanding any provision of this the Parties may, at any time, seek and decide to settle a Dispute.

Judgment upon any award may be entered in any court having jurisdiction.

Recourse to jurisdictions is expressly excluded except as provided for in the ICC Rules of Conciliation and Arbitration."""

miscellaneous = """Amendment contains the entire agreement between the Parties regarding the subject-matter.

Amendment shall not be varied or modified except by a written document duly signed."""

contract = generate_contract(
    contract_sections=[
        definitions,
        amendment,
        effective_date_and_duration,
        confidentiality,
        governing_law,
        miscellaneous,
    ]
)

30/05/24 13:57:19 INFO Prompt:
Bullet points:
Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA.
30/05/24 13:57:38 INFO Raw response:
{
  "thought": "The bullet point provided refers to the definition and interpretation of terms used in the contract. This is typically included in the 'Interpretation' or 'Definitions' section of a contract. I will draft a section that incorporates this point.",
  "tool": "Final Answer",
  "tool_input": {
    "title": "Interpretation and Definitions",
    "content": "For the purposes of this Amendment, all capitalised terms, whether used in singular or plural form, shall have the same meaning as those defined in the General Master Agreement (the 'GMA'). This includes, but is not limited to, terms defined in the body of the GMA, in any schedules, annexes, exhibits or appendices attached thereto, or in any documents incorporated by reference therein. The parties agree that any ambiguity or uncertainty in 

In [24]:
pprint(contract)

('Interpretation and Definitions\n'
 '\n'
 'For the purposes of this Amendment, all capitalised terms, whether used in '
 'singular or plural form, shall have the same meaning as those defined in the '
 "General Master Agreement (the 'GMA'). This includes, but is not limited to, "
 'terms defined in the body of the GMA, in any schedules, annexes, exhibits or '
 'appendices attached thereto, or in any documents incorporated by reference '
 'therein. The parties agree that any ambiguity or uncertainty in the '
 'interpretation of such terms shall be resolved by reference to the GMA. This '
 'provision is intended to ensure consistency and coherence in the '
 'interpretation and application of the terms of this Amendment and the GMA, '
 'and to avoid any potential disputes or misunderstandings that may arise from '
 'differing interpretations of such terms.\n'
 '\n'
 'Amendment to Invoicing and Payment Terms and Price Conditions\n'
 '\n'
 "This Amendment (the 'Amendment') is made to the G

### Sentiment Analysis

For straightforward classification tasks, such as analyzing the sentiment of tweets, we can delegate the labeling process to the LLM. What's neat about this approach is that we can store the LLM's reasoning along with the sentiment label, facilitating the development of explainable AI systems.


In this scenario, it's important to note that this isn't a chat-based application where users manually feed tweets to the LLM one by one and record the results themselves. Instead, the process involves automating the sentiment analysis task, where the LLM classifies the sentiment of tweets without direct user intervention.

In [25]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head(10)

,sentiment,id,date,query,user,tweet
1504206,1,2072130807,Sun Jun 07 20:27:21 PDT 2009,NO_QUERY,AshlynSantino,Why does drama exist in the world? What if it ...
1531655,1,2178077427,Mon Jun 15 07:09:40 PDT 2009,NO_QUERY,Ladymayra,Damm I wish my friend Quincy n my cousin Usval...
1350191,1,2045458556,Fri Jun 05 10:45:22 PDT 2009,NO_QUERY,lifuXD,@ElvaHsiao go get a massage or what! ha at lea...
1023196,1,1882954004,Fri May 22 07:46:03 PDT 2009,NO_QUERY,kayleehawkins,@RockinRita Go a little west to Rochester/Oakl...
160025,0,1956972444,Thu May 28 23:10:04 PDT 2009,NO_QUERY,jomama6881,On my way home n having 2 deal w underage girl...
715607,0,2259469317,Sat Jun 20 17:38:06 PDT 2009,NO_QUERY,thehypemanofnyc,Playin bowlin in new roc. Pops would NOT be pr...
1219050,1,1989894750,Mon Jun 01 03:11:00 PDT 2009,NO_QUERY,christyspanties,"@ashleeadams I know, I appreciate anything you..."
528554,0,2195137071,Tue Jun 16 10:43:43 PDT 2009,NO_QUERY,cynpearls777,It's going to be a long day
1327712,1,2015416002,Wed Jun 03 04:36:17 PDT 2009,NO_QUERY,xdevinnbabyy,"@xFLOYDxMUSICx this girl, she's in my follower..."
1378618,1,2052029188,Fri Jun 05 23:27:17 PDT 2009,NO_QUERY,BabyKJonas,http://twitpic.com/6q8us - Demi Williams


In [26]:
system_prompt = """Take the following tweet and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Sentiment(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet:\n{tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Sentiment,
    iterations=3,
)

In [27]:
def classify_sentiment(tweet: str) -> int:
    response = sentiment_analysis_agent.invoke({'tweet': tweet})
    sentiment_analysis_agent.reset()
    return response.final_answer['sentiment'] or 0

In [28]:
df_tweets_sampled["prediction"] = [classify_sentiment(tweet) for tweet in df_tweets_sampled.tweet]

30/05/24 13:59:18 INFO Prompt:
Tweet:
Why does drama exist in the world? What if it disappeared? :O That would be the day! 
30/05/24 13:59:24 INFO Raw response:
{
  "thought": "The tweet seems to be expressing a desire for a world without drama, which could be interpreted as a positive sentiment. However, it's not a clear-cut positive or negative sentiment as it's more of a hypothetical question or musing rather than a review or opinion.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 1}
}
30/05/24 13:59:24 INFO Thought:
The tweet seems to be expressing a desire for a world without drama, which could be interpreted as a positive sentiment. However, it's not a clear-cut positive or negative sentiment as it's more of a hypothetical question or musing rather than a review or opinion.
30/05/24 13:59:24 INFO Final answer:
{'sentiment': 1}
30/05/24 13:59:24 INFO Prompt:
Tweet:
Damm I wish my friend Quincy n my cousin Usvaldo  were still here with us to celebrate this. But I know th

In [29]:
df_tweets_sampled.head(10)

,sentiment,id,date,query,user,tweet,prediction
1504206,1,2072130807,Sun Jun 07 20:27:21 PDT 2009,NO_QUERY,AshlynSantino,Why does drama exist in the world? What if it ...,1
1531655,1,2178077427,Mon Jun 15 07:09:40 PDT 2009,NO_QUERY,Ladymayra,Damm I wish my friend Quincy n my cousin Usval...,1
1350191,1,2045458556,Fri Jun 05 10:45:22 PDT 2009,NO_QUERY,lifuXD,@ElvaHsiao go get a massage or what! ha at lea...,1
1023196,1,1882954004,Fri May 22 07:46:03 PDT 2009,NO_QUERY,kayleehawkins,@RockinRita Go a little west to Rochester/Oakl...,-1
160025,0,1956972444,Thu May 28 23:10:04 PDT 2009,NO_QUERY,jomama6881,On my way home n having 2 deal w underage girl...,0
715607,0,2259469317,Sat Jun 20 17:38:06 PDT 2009,NO_QUERY,thehypemanofnyc,Playin bowlin in new roc. Pops would NOT be pr...,0
1219050,1,1989894750,Mon Jun 01 03:11:00 PDT 2009,NO_QUERY,christyspanties,"@ashleeadams I know, I appreciate anything you...",0
528554,0,2195137071,Tue Jun 16 10:43:43 PDT 2009,NO_QUERY,cynpearls777,It's going to be a long day,0
1327712,1,2015416002,Wed Jun 03 04:36:17 PDT 2009,NO_QUERY,xdevinnbabyy,"@xFLOYDxMUSICx this girl, she's in my follower...",0
1378618,1,2052029188,Fri Jun 05 23:27:17 PDT 2009,NO_QUERY,BabyKJonas,http://twitpic.com/6q8us - Demi Williams,-1


### Structuring Unstructured Data

An excellent application of LLMs involves organizing unstructured data, such as text documents. Take, for instance, a collection of job descriptions. While there are various methods to tackle this task, like coding a parser or employing optical character recognition, we opt to leverage an LLM for the job. Initially, we define the specific information we wish to extract from the job postings, such as the job title, salary, application instructions, and more. The LLM then looks at each job description and extracts the details for us. Subsequently, we can effortlessly store this organized dataset as a CSV file for further analysis and utilization.

In [30]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()
        jobs.append(content)

With structured output formats, we can now request even more complex data structures. Here, we aim to retrieve strings, integers, and even lists containing strings. Leveraging Pydantic, we can validate the LLM's output. As shown by the dataset below, we can see that the LLM is capable of handling even more complex data structures.

In [31]:
system_prompt = """Take the following job and extract data about the job.

Respond with the job information:
- job title: title of the job.
- job class no: class number.
- open date: when the position was created. Use DD-MM-YYYY.
- salary: the salary ranges.
- deadline: when the application deadline is. Use DD-MM-YYYY.
- application form: online or email or fax.
- where to apply: url or location."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class Job(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: int = Field(description="The job class code.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY")
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job}",
    task_prompt_variables=["job"],
    tools=None,
    output_format=Job,
    iterations=3,
)

In [32]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        response = job_data_agent.invoke({"job": job})
        data.append(response.final_answer)
        job_data_agent.reset()
    return pd.DataFrame(data)

In [33]:
df_jobs = extract_jobs(jobs)

30/05/24 14:00:07 INFO Prompt:
Job description:
BUILDING OPERATING ENGINEER

Class Code:       5923
Open Date:  11-16-18
REVISED:    11-28-18
(Exam Open to All, including Current City Employees)

ANNUAL SALARY

$92,352 and $95,171 (flat-rated)

NOTES:

1. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer. 
2. Some positions may require night and weekend work. Candidates willing to work various shifts are especially desired. Higher salaries are paid for shift work.

DUTIES

A Building Operating Engineer operates, monitors, and performs maintenance on high pressure gas or oil fired boilers, gas or steam turbines, energy recovery boilers and heat exchangers, other heating systems, and their auxiliaries in supplying steam or high temperature hot water for space heating and similar purposes; operates, monitors, and performs maintenance on large tonnage capa

In [34]:
df_jobs.head()

,job_title,job_class_no,job_duties,open_date,salary,deadline,application_form,where_to_apply
0,BUILDING OPERATING ENGINEER,5923,"A Building Operating Engineer operates, monito...",16-11-2018,"[$92,352, $95,171]",06-12-2018,online,https://www.governmentjobs.com/careers/lacity
1,POWER ENGINEERING MANAGER,9453,A Power Engineering Manager may serve as the m...,28-04-2017,"[$145,638 to $180,966, $153,760 to $191,052, $...",11-05-2017,online,https://www.governmentjobs.com/careers/lacity/...
2,ADMINISTRATIVE CLERK,1358,An Administrative Clerk performs general offic...,30-03-2018,"[$37,584 to $54,935, $43,785 (flat rated); $43...",12-04-2018,online,https://www.governmentjobs.com/careers/lacity
3,SENIOR ELECTRICAL TEST TECHNICIAN,7515,A Senior Electrical Test Technician performs h...,29-09-2017,"[$81,703 to $101,497, $106,446 to $112,376, $1...",19-10-2017,online,https://www.governmentjobs.com/careers/lacity/...
4,STORES SUPERVISOR,1866,"A Stores Supervisor plans, organizes and direc...",29-12-2017,"[$84,376 to $120,039, $101,247 to $125,760]",11-01-2018,online,https://www.governmentjobs.com/careers/lacity/...


## 5. Multi-Agents

When tasks assigned to an LLM become too complex, it is essential to divide the AI agent into multiple components. The common approach is to split the AI agent into multiple agents with specific goals, either sequentially chaining them or linking them in a graph-like structure. This is one way to let multiple LLMs collaborate to solve a specific problem.

### Comparing Unstructured Data

When comparing two job descriptions, one approach involves presenting both job postings to an LLM to identify similarities and differences. However, this method may not yield optimal results, as job descriptions often include non-essential information such as "equal employment opportunity" statements. To address this, we can deconstruct the problem by initially tasking an LLM to extract relevant data from each job description. In our case 2 instances as we are comparing 2 jobs. Subsequently, this condensed information can be inputted into a 3rd LLM, tasked with analyzing the extracted data to identify differences and similarities between the two job postings.

<center>

| Input: |
|------|
| Job 1 |
| Job 2 |

</center>

<p align="center">
  &darr;
</p>

<center>

| LLM that extracts job data: |
|------|
| Input: Job 1 |
| Output: Job title 1, job duties 1, salary 1 |

</center>

<p align="center">
  &darr;
</p>

<center>

| LLM that extracts job data: |
|------|
| Input: Job 2 |
| Output: Job title 2, job duties 2, salary 2 |

</center>

<center>
  &darr;
</center>

<center>

| LLM that compares 2 jobs: |
|------|
| Input: Job title 1, job duties 1, salary 1, job title 2, job duties 2, salary 2 |
| Output: Differences, similarities |

</center>

<p align="center">
  &darr;
</p>

<center>

| Output: |
|------|
| Differences |
| Similarities |

</center>

In [35]:
def get_job(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()
        return content

job1 = get_job("./data/jobs/ELECTRICAL ENGINEERING ASSOCIATE 7525 093016 REV 100416.txt")
job2 = get_job("./data/jobs/ELECTRICAL MECHANIC 3841 012017.txt")

In [36]:
system_prompt = "Take the following job and extract data about the job"

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class Job1(BaseModel):
    job1_title: str = Field(description="The job title.")
    job1_duties: str = Field(description="The duties of the job.")
    salary1: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

class Job2(BaseModel):
    job2_title: str = Field(description="The job title.")
    job2_duties: str = Field(description="The duties of the job.")
    salary2: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

job_agent1 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job1}",
    task_prompt_variables=["job1"],
    tools=None,
    output_format=Job1,
    iterations=10,
)

job_agent2 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job2}",
    task_prompt_variables=["job2"],
    tools=None,
    output_format=Job2,
    iterations=10,
)

In [37]:
system_prompt = "Take the following 2 job descriptions and respond with the similarities and differences of the jobs."

task_prompt = """Compare the 2 given job descriptions:

Job 1:
Job title: {job1_title}
Job duties:
{job1_duties}
Salary:
{salary1}


Job 2:
Job title: {job2_title}
Job duties:
{job2_duties}
Salary:
{salary2}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class JobComparison(BaseModel):
    similarities: str = Field(description="The job similarities.")
    differences: str = Field(description="The job differences.")

job_comparison_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["job1_title", "job1_duties", "salary1", "job2_title", "job2_duties", "salary2"],
    tools=None,
    output_format=JobComparison,
    iterations=10,
)

In [38]:
chain = AgentChain(
    chain=[job_agent1, job_agent2, job_comparison_agent],
    chain_variables=["job1", "job2"],
)

In [39]:
response = chain.invoke({"job1": job1, "job2": job2})

30/05/24 14:04:31 INFO Prompt:
Job description:
ELECTRICAL ENGINEERING ASSOCIATE
Class Code:       7525
Open Date:  09-30-16
REVISED: 10-04-16
 (Exam Open to All, including Current City Employees)
ANNUAL SALARY 

$66,231 to $94,252; $74,082 to $105,444; $82,497 to $117,346; and $89,638 to $127,556
The salary in the Department of Water and Power is $77,360 to $96,110; $91,934 to $114,213; $99,722 to $123,881; and $107,156 to 
$133,130

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. For information regarding reciprocity between City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
3. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

An Electrical Engineering Associate performs professional electrical engineering work in the preparation of designs, plans, specifications

In [40]:
pprint(response.final_answer["similarities"])

('Both jobs are in the electrical field and involve working with electrical '
 'systems and equipment. They both require technical knowledge and skills in '
 'electrical systems.')


In [41]:
pprint(response.final_answer["differences"])

('The Electrical Engineering Associate is more focused on design, planning, '
 'and quality assurance, while the Electrical Mechanic is more hands-on, '
 'dealing with the installation and maintenance of electrical circuits and '
 'equipment. The salary range for the Electrical Engineering Associate is '
 'wider and generally higher than that of the Electrical Mechanic.')


### Machine Learning Code Generation

In [42]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [43]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [44]:
ml_chain = AgentChain(
    chain=[problem_finder_agent, ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [45]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [46]:
response = ml_chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.",
    "dataset_size": len(df_tweets),
    "dataset_schema": dataset_schema,
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

30/05/24 14:05:29 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

30/05/24 14:05:38 INFO Raw response:
{
  "thought": "Given the problem description and the dataset, it seems like the user wants to predict the sentiment of tweets, which is a categorical variable. This is a typical example of a classification problem.",
  "tool": "Final Answer",
 

In [47]:
print(response.final_answer["code"])

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Load data
# df = pd.read_csv('data.csv')

# Preprocess text data in 'tweet' column
# df['tweet'] = df['tweet'].apply(preprocess_text)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2, random_state=42)

# Vectorize 'tweet' data
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

# Make predictions on test set
y_pred = clf.predict(X_test_vec)

# Evaluate model
print(classification_report(y_test, y_pred))


### Forecasting

In [48]:
from datetime import timedelta
from language_models.tools.forecasting import get_earthquakes_data, ml_model

In [49]:
class Forecast(BaseModel):
    start_time: str = Field(None, description='Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')
    end_time: str = Field(None, description='Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')

def forecast(start_time = None, end_time = None):
    if start_time is None:
        start_time = (datetime.now() - timedelta(days=30)).date()
    if end_time is None:
        end_time = (datetime.now().date())
    df = get_earthquakes_data('https://earthquake.usgs.gov/fdsnws/event/1/query?', start_time, end_time)
    df_pred = ml_model.predict(df)
    return {'predictions': df_pred.to_dict(orient='records')}

In [50]:
forecasting_tool = Tool(func=forecast, name='Forecast', description='Test forecast model on real-time events.', args_schema=Forecast)

In [51]:
task_prompt = """{question}

Use the Current Date tool and respond with the start time and end time."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    float=0.0,
)

class DateRange(BaseModel):
    start_time: str = Field(description="The start date. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")
    end_time: str = Field(description="The end date. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[current_date_tool],
    output_format=DateRange,
    iterations=5,
)

In [52]:
forecast_chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool],
    chain_variables=["question"],
)

In [53]:
response = forecast_chain.invoke({"question": "Run a forecast using the past week as data."})

30/05/24 14:06:28 INFO Prompt:
Run a forecast using the past week as data.

Use the Current Date tool and respond with the start time and end time.
30/05/24 14:06:36 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the past week's dates.",
  "tool": "Current Date",
  "tool_input": ""
}
30/05/24 14:06:41 INFO Raw response:
{
  "thought": "The user wants to run a forecast using the past week as data. I need to use the Current Date tool to get the current date and then calculate the start time and end time for the past week.",
  "tool": "Current Date",
  "tool_input": "{}"
}
30/05/24 14:06:47 INFO Raw response:
{
  "thought": "I need to use the Current Date tool to get the current date and time. Then, I will calculate the start time and end time for the past week. However, the tool doesn't require any input.",
  "tool": "Current Date",
  "tool_input": "{}"
}
30/05/24 14:06:52 INFO Raw response:
{
  "thought": "I need to use the Current Date tool to get

In [54]:
print(response.final_answer["Forecast"])

{'predictions': [{'time': Timestamp('2024-05-23 14:08:18.120000+0000', tz='UTC'), 'prediction': 1.94539, 'latitude': 38.8156661987305, 'longitude': -122.86483001709, 'mag': 0.77, 'id': 'nc75011026', 'place': '10 km NW of The Geysers, CA', 'location': '10 km NW of The Geysers, CA'}, {'time': Timestamp('2024-05-23 14:10:35.880000+0000', tz='UTC'), 'prediction': 2.325778, 'latitude': 33.001, 'longitude': -116.3686667, 'mag': 1.24, 'id': 'ci40597455', 'place': '23 km ESE of Julian, CA', 'location': '23 km ESE of Julian, CA'}, {'time': Timestamp('2024-05-23 14:19:14.630000+0000', tz='UTC'), 'prediction': 1.91261, 'latitude': 37.5191666666667, 'longitude': -118.8755, 'mag': 0.47, 'id': 'nc75011031', 'place': '16 km SSE of Mammoth Lakes, CA', 'location': '16 km SSE of Mammoth Lakes, CA'}, {'time': Timestamp('2024-05-23 14:30:12.309000+0000', tz='UTC'), 'prediction': 4.180065, 'latitude': 41.7853, 'longitude': 82.4865, 'mag': 4.8, 'id': 'us6000n0g7', 'place': '37 km WNW of Kuqa, China', 'locat

### LLM-powered Functions

AI agents can team up with other AI agents. When an AI agent is put into a function that gets instructions, it becomes a tool, just like any other function in code. Their true power lies in being equipped with tools. By leveraging this, we can create a network of interconnected capabilities, where AI agents use tools backed by other AI agents.

In [55]:
earthquake_agent.reset()
problem_finder_agent.reset()
ml_agent.reset()

In [56]:
class EarthquakeAgent(BaseModel):
    question: str = Field(description="The question regarding earthquakes.")

def answer_earthquake_questions(question: str) -> Any:
    response = earthquake_agent.invoke({"question": question})
    return response.final_answer

earthquake_agent_tool = Tool(
    func=answer_earthquake_questions,
    name="Earthquake Agent",
    description="Use this tool to answer questions about earthquakes.",
    args_schema=EarthquakeAgent,
)

class MLAgent(BaseModel):
    problem_description: str = Field(description="The user problem.")
    dataset_size: int = Field(description="The size of the dataset."),
    dataset_schema: str = Field(description="The dataset schema or information."),
    dataset_snippet: str = Field(description="The dataset snippet aka the first couple of rows of the dataset.")

def generate_ml_code(problem_description: str, dataset_size: int, dataset_schema: str, dataset_snippet: str) -> Any:
    response = ml_chain.invoke({
        "problem_description": problem_description,
        "dataset_size": dataset_size,
        "dataset_schema": dataset_schema,
        "dataset_snippet": dataset_snippet,
    })
    return response.final_answer

ml_agent_tool = Tool(
    func=generate_ml_code,
    name="ML Agent",
    description="Use this tool to generate machine learning code given a problem.",
    args_schema=MLAgent,
)

In [57]:
system_prompt = """You are an Agent that delegates tasks to other Agents by using the appropriate tools.

Use the Earthquake Agent when the question is about earthquakes.

Use the ML Agent when the user wants you to generate machine learning code."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

almighty_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{prompt}",
    task_prompt_variables=["prompt"],
    tools=[earthquake_agent_tool, ml_agent_tool],
    output_format=Output,
    iterations=10,
)

/opt/homebrew/Caskroom/miniforge/base/envs/llm-powered-ai-agents/lib/python3.11/site-packages/pydantic/json_schema.py:2158: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The size of the dataset.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/llm-powered-ai-agents/lib/python3.11/site-packages/pydantic/json_schema.py:2158: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The dataset schema or information.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [58]:
response = almighty_agent.invoke({"prompt": "How many earthquakes happened today?"})

30/05/24 14:07:07 INFO Prompt:
How many earthquakes happened today?
30/05/24 14:07:13 INFO Raw response:
{
  "thought": "The user is asking about earthquakes. I should use the Earthquake Agent to get this information.",
  "tool": "Earthquake Agent",
  "tool_input": {"question": "How many earthquakes happened today?"}
}
30/05/24 14:07:13 INFO Thought:
The user is asking about earthquakes. I should use the Earthquake Agent to get this information.
30/05/24 14:07:13 INFO Tool:
Earthquake Agent
30/05/24 14:07:13 INFO Tool input:
{'question': 'How many earthquakes happened today?'}
30/05/24 14:07:13 INFO Prompt:
How many earthquakes happened today?
30/05/24 14:07:18 INFO Raw response:
{
  "thought": "I need to get the current date first to determine the start and end times for the earthquake query.",
  "tool": "Current Date",
  "tool_input": {}
}
30/05/24 14:07:18 INFO Thought:
I need to get the current date first to determine the start and end times for the earthquake query.
30/05/24 14:07

In [59]:
print(response.final_answer["content"])

There were 89 earthquakes today.


In [60]:
prompt = f"""Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: {len(df_tweets)}
Schema:
{dataset_schema}
First 10 rows of dataset:
{df_tweets.head(10).to_markdown()}"""

response = almighty_agent.invoke({"prompt": prompt})

30/05/24 14:07:40 INFO Prompt:
Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

First 10 rows of dataset:
|    |   sentiment |         id | date                         | query    | user            | tweet                                                                                                               |
|---:|------------:|-----------:|:-----------------------------|:---------|:----------------|:-----------------------------------

In [61]:
print(response.final_answer["content"])

Here is the Python code to train a model that predicts the sentiment of a tweet:

```python
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load the data
# df = pd.read_csv('data.csv')

# Convert the text data into numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['tweet'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)

# Train the model
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Model accuracy: ', accuracy)
```

Please replace 'data.csv' with your actual data file path.
